## Machine learning (Logistic Regresion)

In [561]:
import os
import pandas as pd

# Path donde se encuentran mis archivos train
BASE_PATH = "./data/processed"
TRAIN_PATHS = [
    "X_train_con_outliers.xlsx",
    "X_train_sin_outliers.xlsx",
    "X_train_con_outliers_norm.xlsx",
    "X_train_sin_outliers_norm.xlsx",
    "X_train_con_outliers_scal.xlsx",
    "X_train_sin_outliers_scal.xlsx"
]

# Guardo cada uno de estos archivos dentro de una lista
TRAIN_DATASETS = []
for path in TRAIN_PATHS:
    TRAIN_DATASETS.append(
        # pd.read_excel(BASE_PATH + "/" + path)
        pd.read_excel(f"{BASE_PATH}/{path}")
        # pd.read_excel(os.path.join(BASE_PATH, path))
    )

# Path donde se encuentran mis archivos test
TEST_PATHS = [
    "X_test_con_outliers.xlsx",
    "X_test_sin_outliers.xlsx",
    "X_test_con_outliers_norm.xlsx",
    "X_test_sin_outliers_norm.xlsx",
    "X_test_con_outliers_scal.xlsx",
    "X_test_sin_outliers_scal.xlsx"
]

# Guardo cada uno de estos archivos dentro de una lista
TEST_DATASETS = []
for path in TEST_PATHS:
    TEST_DATASETS.append(
        pd.read_excel(f"{BASE_PATH}/{path}")
    )

y_train = pd.read_excel(f"{BASE_PATH}/y_train.xlsx")
y_test = pd.read_excel(f"{BASE_PATH}/y_test.xlsx")

## 

In [562]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

results = []
for index, dataset in enumerate(TRAIN_DATASETS):
    print(index)
    model = LogisticRegression(random_state = 42, class_weight='balanced')
    model.fit(dataset, y_train)
    y_pred_train = model.predict(dataset)
    y_pred_test = model.predict(TEST_DATASETS[index])

    results.append(
        {
            "train": accuracy_score(y_train, y_pred_train),
            "test": accuracy_score(y_test, y_pred_test)
        }
    )

results

0
1
2
3


C:\Users\Carlos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Carlos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Carlos\AppData\Lo

4
5


C:\Users\Carlos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Carlos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[{'train': 0.7220637329286799, 'test': 0.7177713037144938},
 {'train': 0.7207587253414264, 'test': 0.7164360281621753},
 {'train': 0.7358421851289834, 'test': 0.7320951687302744},
 {'train': 0.7207890743550834, 'test': 0.7171643602816218},
 {'train': 0.7353869499241275, 'test': 0.7310026705511047},
 {'train': 0.7207890743550834, 'test': 0.7171643602816218}]

En este caso el que tiene una mayor precision es que tiene indice 2

Primero una buscado de manera aleatoria general para encontrar los mejores valores y optimizar sobre ellos posteriormente

In [594]:
from sklearn.model_selection import RandomizedSearchCV

best_dataset = 2


hyperparams_general = {
    'C': np.logspace(-3, 3, 1000),          # Desde regularización muy fuerte hasta muy débil
    'tol': np.linspace(0.0001, 0.01, 500),    # Rango amplio para el criterio de convergencia
    'fit_intercept': [True, False],
    'penalty': ['l1', 'l2'],                # Simplificamos a l1 y l2 para el ejemplo
    'max_iter': [100, 500, 1000],
    'solver': ['saga']                     # 'saga' es flexible y soporta ambas penalizaciones
}

model = LogisticRegression(random_state=42, class_weight='balanced')
grid_general  = RandomizedSearchCV(model, hyperparams_general, n_iter=20, cv=10, scoring='f1', random_state=42, n_jobs=-1)

In [595]:
grid_general.fit(TRAIN_DATASETS[best_dataset], y_train)

print("Mejor score en RandomizedSearchCV:", grid_general.best_score_)
print("Mejores parámetros encontrados:", grid_general.best_params_)

Mejor score en RandomizedSearchCV: 0.37268205381915454
Mejores parámetros encontrados: {'tol': np.float64(0.009702404809619238), 'solver': 'saga', 'penalty': 'l1', 'max_iter': 100, 'fit_intercept': True, 'C': np.float64(85.29644499741016)}


C:\Users\Carlos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [596]:
# Extraemos los mejores parámetros de la búsqueda amplia
best_params = grid_general.best_params_
best_C = best_params["C"]
best_tol = best_params["tol"]

# Creamos un grid refinado en torno a los mejores valores encontrados
# Por ejemplo, generamos 10 valores para C en un rango pequeño alrededor del óptimo
C_range = np.logspace(np.log10(best_C) - 0.5, np.log10(best_C) + 0.5, 10)
# Y para tol, usamos un rango lineal ajustado alrededor del valor óptimo
tol_range = np.linspace(max(best_tol - 0.0005, 0.0001), best_tol + 0.0005, 10)

# Creamos el grid refinado, fijando los demás parámetros al mejor encontrado
hyperparams_optimizacion = {
    'C': C_range,
    'tol': tol_range,
    'fit_intercept': [best_params['fit_intercept']],
    'penalty': [best_params['penalty']],
    'max_iter': [best_params['max_iter']],
    'solver': [best_params['solver']]
}

# Configuramos GridSearchCV para refinar la búsqueda
grid_optimized = GridSearchCV(model, hyperparams_optimizacion, cv=10, scoring='f1', n_jobs=-1)

# Ajustamos el modelo con el grid refinado
grid_optimized.fit(TRAIN_DATASETS[best_dataset], y_train)

print("Mejor score en GridSearchCV refinado:", grid_optimized.best_score_)
print("Mejores parámetros refinados:", grid_optimized.best_params_)


Mejor score en GridSearchCV refinado: 0.37268205381915454
Mejores parámetros refinados: {'C': np.float64(26.97310425070911), 'fit_intercept': True, 'max_iter': 100, 'penalty': 'l1', 'solver': 'saga', 'tol': np.float64(0.009202404809619237)}


C:\Users\Carlos\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [597]:
from sklearn.metrics import accuracy_score

final_model = grid_optimized.best_estimator_
y_pred_train = grid_optimized.predict(TRAIN_DATASETS[best_dataset])
y_pred_test = grid_optimized.predict(TEST_DATASETS[best_dataset])

In [598]:
# 0.8888922610015174
train_accuracy = accuracy_score(y_train, y_pred_train)
train_accuracy

# 0.8889529590288315


0.7356904400606981

In [599]:
# 0.8860160233066279
test_accuracy = accuracy_score(y_test, y_pred_test)
test_accuracy

# 0.8866229667394999


0.7318523913571255

Dudas:

- que hemos hecho con feature selection, no lo hemos usado?
- Por que apenas obtengo mejoras y como puedo conseguir llegar al 0.9
- Hay alguna manera de poder graficar la progresion de los hyperparametros cuando se calcula con un conjunto de estos? Lo que yo he hecho los calcula individualmente y no me da buen resultado
- En cual de los valores me tengo que fijar?

- He visto que en casos como estos en los que las clases estan desbalanceadas(muchos mas no que si), la metrica accuracy no es correcta y puede estar fallando y dando falsos positivos.
Que otras metricas tendria que utilizar?

In [600]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score

# Asumiendo que ya tienes y_test y y_pred (y_prob si calculas ROC AUC)
acc = accuracy_score(y_test, y_pred_test)
prec = precision_score(y_test, y_pred_test)
rec = recall_score(y_test, y_pred_test)
f1 = f1_score(y_test, y_pred_test)
cm = confusion_matrix(y_test, y_pred_test)


print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1 Score:", f1)
print("Confusion Matrix:\n", cm)

Accuracy: 0.7318523913571255
Precision: 0.25175370226032734
Recall: 0.6909090909090909
F1 Score: 0.36903741788060557
Confusion Matrix:
 [[5383 1920]
 [ 289  646]]


In [605]:
y_prob = final_model.predict_proba(TEST_DATASETS[best_dataset])[:, 1]

# Suponiendo que decidimos probar un umbral de 0.4 en lugar del 0.5
umbral = 0.4
y_pred_ajustado = (y_prob > umbral).astype(int)

# Calculamos las métricas con el nuevo umbral
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
print("Accuracy:", accuracy_score(y_test, y_pred_ajustado))
print("Precision:", precision_score(y_test, y_pred_ajustado))
print("Recall:", recall_score(y_test, y_pred_ajustado))
print("F1 Score:", f1_score(y_test, y_pred_ajustado))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_ajustado))

Accuracy: 0.7076960427288177
Precision: 0.2374331550802139
Recall: 0.7122994652406417
F1 Score: 0.35614973262032085
Confusion Matrix:
 [[5164 2139]
 [ 269  666]]


Conclusiones:

El modelo tiene un entrenamiento correcto, dependiendo del objetivo del negocio podemos cambiar el umbral de esta manera:

 - Con un umbral de 0.4, se detecta a más clientes (recall del 71%), pero la precisión cae a alrededor del 24%, lo que implica que se contactará a muchos clientes que no son realmente interesados. El F1 Score es ligeramente inferior al de la configuración por defecto(0.5).

 - Con un el umbral de 0.6, el modelo se vuelve más conservador. Esto eleva la precisión (más de 32% de las predicciones positivas son correctas) y mejora el F1 Score, pero reduce el recall a alrededor del 58%, lo que significa que se pierden algunos clientes potenciales.

 - Con el umbral por defecto se capturan aproximadamente el 69% de los verdaderos positivos, pero solo el 25% de los clientes contactados realmente son potenciales interesados.


En principio la manera de captar mas clientes seria con un umbral del 0.4 ya que se contacta a muchos mas clientes, pero conlleva un mayor gasto economico ya que es menos preciso.